In [1]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import Image, display

In [2]:
!pip install ultralytics

In [3]:
!pip install albumentations
!pip install opencv-python


# Data preprocessing Train

In [5]:

def preprocess_image(image_path, output_path, img_size=(640,640)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, img_size)
    img = img /255.0
    cv2.imwrite(output_path, (img * 255).astype(np.uint8))



input_dir = r'C:\Users\USER\Desktop\new.v1i.yolov8\train\images'
output_dir = r'C:\Users\USER\Desktop\por'
    
    
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for image_name in os.listdir(input_dir):
    preprocess_image(os.path.join(input_dir, image_name), os.path.join(output_dir, image_name))

# Data preprocessing Val

In [6]:

input_dir = r'C:\Users\USER\Desktop\new.v1i.yolov8\valid\images'
output_dir = r'C:\Users\USER\Desktop\por2'
    
    
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for image_name in os.listdir(input_dir):
    preprocess_image(os.path.join(input_dir, image_name), os.path.join(output_dir, image_name))

# image  Augmentation

### Function

In [27]:
augmentation_pipeline = A.Compose([                    
    A.RandomCrop(width=600, height=600),    # Random cropping
    A.HorizontalFlip(p=0.5),                 # Horizontal flip with 50% probability
    A.Rotate(limit=30, p=0.5),             # Random rotation within 30 degrees
    A.RandomBrightnessContrast(p=0.5),       # Random brightness and contrast adjustment
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.5),  # Scaling and translation
    ToTensorV2()                            
])


def augment_and_save_images(input_folder, output_folder):
    
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
   
    for filename in tqdm(os.listdir(input_folder)):
        
        image_path = os.path.join(input_folder, filename)
        
       
        image = cv2.imread(image_path)
        if image is None:
            print(f"Skipping file {filename}, as it is not a valid image.")
            continue
        
       
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        
        augmented = augmentation_pipeline(image=image)
        augmented_image = augmented['image']
        
        
        augmented_image_np = augmented_image.permute(1, 2, 0).cpu().numpy()
        
        
        augmented_image_bgr = cv2.cvtColor(augmented_image_np, cv2.COLOR_RGB2BGR)
        
        
        output_path = os.path.join(output_folder, f"aug_{filename}")
        
        
        cv2.imwrite(output_path, augmented_image_bgr)

### call 

In [26]:
input_folder = r'C:\Users\USER\Desktop\old'  
output_folder = r'C:\Users\USER\Desktop\augmented'  

augment_and_save_images(input_folder, output_folder)

100%|██████████| 5/5 [00:00<00:00, 91.60it/s]


### Create a Ymal file 

## Yolov8 Small

In [39]:
from ultralytics import YOLO

# Load a YOLOv8 Small model from scratch
model = YOLO("yolov8s.yaml")
model = YOLO("yolov8s.pt")

100%|██████████| 21.5M/21.5M [00:14<00:00, 1.57MB/s]


## Yolov8 Nano

In [37]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

FileNotFoundError: 'yolov8S.yaml' does not exist

### Hyperparameter Tuning

In [35]:
hyperparameters = {
    'lr0': 0.01,  # initial learning rate
    'lrf': 0.1,   # final learning rate (lr0 * lrf)
    'momentum': 0.937,  # SGD momentum
    'weight_decay': 0.0005,  # optimizer weight decay
    'warmup_epochs': 3.0,  # warmup period
    'warmup_momentum': 0.8,  # warmup initial momentum
    'warmup_bias_lr': 0.1,  # warmup initial bias lr
    'box': 0.05,  # box loss gain
    'cls': 0.5,  # cls loss gain
    'iou': 0.2,  # IoU training threshold
    'hsv_h': 0.015,  # image HSV-Hue augmentation (fraction)
    'hsv_s': 0.7,  # image HSV-Saturation augmentation (fraction)
    'hsv_v': 0.4,  # image HSV-Value augmentation (fraction)
    'degrees': 0.0,  # image rotation (+/- deg)
    'translate': 0.1,  # image translation (+/- fraction)
    'scale': 0.5,  # image scale (+/- gain)
    'shear': 0.0,  # image shear (+/- deg)
    'perspective': 0.0,  # image perspective (+/- fraction), range 0-0.001
    'flipud': 0.0,  # image flip up-down (probability)
    'fliplr': 0.5,  # image flip left-right (probability)
    'mosaic': 1.0,  # image mosaic (probability)
    'mixup': 0.0,  # image mixup (probability)
    'copy_paste': 0.0,  # segment copy-paste (probability)
}

In [40]:
#, **hyperparameters
# Use the model
model.train(data="data.yaml", epochs=30)  # train the model
metrics = model.val()  # evaluate model performance on the validation set


New https://pypi.org/project/ultralytics/8.2.61 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58  Python-3.12.3 torch-2.3.1+cpu CPU (13th Gen Intel Core(TM) i7-13700F)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning C:\Users\USER\Desktop\1\new.v1i.yolov8\train\labels.cache... 115 images, 0 backgrounds, 15 corrupt: 100%|██████████| 115/115 [00:00<?, ?it/s]

train: WARNING  C:\Users\USER\Desktop\1\new.v1i.yolov8\train\images\aug_img3_png.rf.9d5c6787463d8a1809b3ba9bf693c734.jpg: ignoring corrupt image/label: Label class 15 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING  C:\Users\USER\Desktop\1\new.v1i.yolov8\train\images\aug_img5_png.rf.6f6cd41f80d90298bdc8c14c00702199.jpg: ignoring corrupt image/label: Label class 15 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING  C:\Users\USER\Desktop\1\new.v1i.yolov8\train\images\aug_img6_png.rf.a677ba4461334b0d94c30f13d7c6d2ad.jpg: ignoring corrupt image/label: Label class 16 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING  C:\Users\USER\Desktop\1\new.v1i.yolov8\train\images\aug_img7_png.rf.6929b96d170242949373189ed9bf147b.jpg: ignoring corrupt image/label: Label class 15 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING  C:\Users\USER\Desktop\1\new.v1i.yolov8\train\images\aug_img8_png.rf.d6e9


val: Scanning C:\Users\USER\Desktop\1\new.v1i.yolov8\valid\labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]

Plotting labels to runs\detect\train9\labels.jpg... 


...


30 epochs completed in 0.291 hours.
Optimizer stripped from runs\detect\train9\weights\last.pt, 22.5MB
Optimizer stripped from runs\detect\train9\weights\best.pt, 22.5MB

Validating runs\detect\train9\weights\best.pt...
Ultralytics YOLOv8.2.58  Python-3.12.3 torch-2.3.1+cpu CPU (13th Gen Intel Core(TM) i7-13700F)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.53s/it]


                   all         25         97      0.649      0.559      0.613      0.293
             With_Mask         24         89      0.846      0.742      0.812      0.332
          Without_Mask          7          8      0.451      0.375      0.414      0.255
Speed: 0.9ms preprocess, 93.1ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\train9
Ultralytics YOLOv8.2.58  Python-3.12.3 torch-2.3.1+cpu CPU (13th Gen Intel Core(TM) i7-13700F)
Model summary (fused): 168 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning C:\Users\USER\Desktop\1\new.v1i.yolov8\valid\labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.23s/it]


                   all         25         97      0.649      0.559      0.613      0.293
             With_Mask         24         89      0.846      0.742      0.812      0.332
          Without_Mask          7          8      0.451      0.375      0.414      0.255
Speed: 1.5ms preprocess, 88.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs\detect\train92


In [9]:
path = model.export(format="onnx")  # export the model to ONNX format

Ultralytics YOLOv8.2.58  Python-3.12.3 torch-2.3.1+cpu CPU (13th Gen Intel Core(TM) i7-13700F)

PyTorch: starting from 'runs\detect\train73\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
Retry 1/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0" ' returned non-zero exit status 1.
Retry 2/2 failed: Command 'pip install --no-cache-dir "onnx>=1.12.0" ' returned non-zero exit status 1.
requirements:  Command 'pip install --no-cache-dir "onnx>=1.12.0" ' returned non-zero exit status 1.
ONNX: export failure  109.9s: No module named 'onnx.defs'


ModuleNotFoundError: No module named 'onnx.defs'